# bottleneck

In [ ]:
# check them out
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.applications.vgg16 import preprocess_input
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, GlobalMaxPooling2D, Flatten
from keras import backend as K

import numpy as np
import math

In [ ]:
PATH_TO_DF = "../data/output_convnet/Inception"

img_width = 150
img_height = 150

n_epochs = 50
batch_size = 16

train_data_dir = "train"  
validation_data_dir = "validation"
test_data_dir = "test"

n_train_samples = 3000
n_validation_samples = 1000
n_test_samples = 1000
n_classes = 8

In [ ]:
# https://b/log.keras.io/building-powerful-image-classification-models-using-very-little-data.html
# saving bottleneck features
import time

ti_bn_features = time.time()

model = InceptionV3(include_top=False, weights='imagenet')

datagen = ImageDataGenerator(rescale=1. / 255)

'''
https://github.com/keras-team/keras/blob/master/keras/preprocessing/image.py#L1002
if PIL version 1.1.3 interpolation = 'lanczos'
else interpolation = 'bicubic' 
''' 

# train
generator = datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    shuffle=False,
    class_mode = None,
    interpolation = 'lanczos')

# important step in order to get the exact number
max_queue_size_train = int(math.ceil(n_train_samples / batch_size))

bnfeatures_train = model.predict_generator(
    generator, max_queue_size_train)

np.save('../data/output_convnet/Inception/inception_bnfeatures_train_aux.npy', bnfeatures_train)

# ref attribute classes --> https://keras.io/preprocessing/image/
train_labels = generator.classes # the key attribute
train_labels = to_categorical(train_labels, num_classes=n_classes) # the key function

# validation
generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_width, img_height),
    batch_size=batch_size,
    shuffle=False,
    class_mode = None,
    interpolation = 'lanczos')

max_queue_size_val = int(math.ceil(n_validation_samples / batch_size))


bnfeatures_val = model.predict_generator(  
     generator, max_queue_size_val)

np.save('../data/output_convnet/Inception/inception_bnfeatures_val_aux.npy', bnfeatures_val)

val_labels = generator.classes # the key attribute
val_labels = to_categorical(val_labels, num_classes=n_classes) # the key function


tf_bn_features = time.time()    
tt_bn_features = tf_bn_features - ti_bn_features

In [ ]:
print(time.strftime("%H:%M:%S", time.gmtime(tt_bn_features)))

In [ ]:
# training top layer
import os

ti_bn_train = time.time()

train_data = np.load('../data/output_convnet/Inception/inception_bnfeatures_train_aux.npy')
val_data = np.load('../data/output_convnet/Inception/inception_bnfeatures_val_aux.npy') 

# top model, could be with a diff dense, optimizer, momentum -> https://keras.io/optimizers/
model = Sequential()  
model.add(GlobalAveragePooling2D(input_shape=train_data.shape[1:]))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(n_classes, activation='softmax'))  

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9),
              loss='categorical_crossentropy', metrics=['accuracy']) 


historical_data = model.fit(train_data, train_labels,
                    epochs=n_epochs,
                    batch_size=batch_size,
                    validation_data=(val_data, val_labels))  

# h5py
model.save_weights('../data/output_convnet/Inception/inception_bn_model_aux.h5')  

tf_bn_train = time.time()    
tt_bn_train = tf_bn_train - ti_bn_train

In [ ]:
print(time.strftime("%H:%M:%S", time.gmtime(tt_bn_train)))

In [ ]:
(loss, acc) = model.evaluate(val_data, val_labels, batch_size=batch_size, verbose=0)
print("acc: {0:.2f}% - loss: {1:f}".format(acc * 100, loss))  

In [ ]:
# historical_data.history acc and loss data over the epochs (train and validation)
import matplotlib.pyplot as plt

train_acc = historical_data.history['acc']
train_loss = historical_data.history['loss']

val_acc = historical_data.history['val_acc'] # validation
val_loss = historical_data.history['val_loss'] # validation

range_epochs = range(n_epochs)

fig = plt.figure(figsize=(20, 6))

fig.add_subplot(1,2,1)
plt.plot(range_epochs, train_acc, 'bo', label='Training acc')
plt.plot(range_epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

# plt.figure()
fig.add_subplot(1,2,2)
plt.plot(range_epochs, train_loss, 'bo', label='Training loss')
plt.plot(range_epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.savefig(os.path.join(PATH_TO_DF, "inception_bn_acc_loss.png"), bbox_inches='tight') # png 70kb vs jpg 135 kb
plt.show()

# fine tuning

In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input

In [ ]:
top_model_weights_path = '../data/output_convnet/Inception/inception_bn_model_aux.h5'
batch_size = 5 # con 5 funciona con 16 es imposible probar con 10

In [ ]:
# base model
input_tensor = Input(shape=(img_width,img_height,3)) # another way to shape the input
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=input_tensor)

In [ ]:
# top model
top_model = Sequential()
top_model.add(GlobalMaxPooling2D(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(1024, activation='relu'))
top_model.add(Dense(8, activation='softmax'))

In [ ]:
# base model has its weights, now we load the weights on the top layer
top_model.load_weights(top_model_weights_path)

In [ ]:
# we join base and top it has to be updated to api2
model_total = Model(input= base_model.input, output= top_model(base_model.output))

In [ ]:
# sequential is the top layer a complex one
for i, layer in enumerate(model_total.layers):
    print (i, layer.name, layer.output_shape)

In [ ]:
for i, layer in enumerate(model_total.layers):
    if layer.trainable:
        print("layer {0:d}, {1:s} is trainable".format(i, layer.name))
    else:
        print("layer {0:d}, {1:s} is freezed".format(i, layer.name))

In [ ]:
# freezing layers implies they will not update their weights over the training
# https://github.com/keras-team/keras/blob/master/keras/applications/inception_v3.py#L325
# tha's the key to the last conv layer
for layer in model_total.layers[:280]:
    layer.trainable = False
for layer in model_total.layers[280:]:
    layer.trainable = True

In [ ]:
# check for updates
for i, layer in enumerate(model_total.layers):
    if layer.trainable:
        print("layer {0:d}, {1:s} is trainable".format(i, layer.name))
    else:
        print("layer {0:d}, {1:s} is freezed".format(i, layer.name))

In [ ]:
mmodel_total.compile(optimizer=SGD(lr=1e-4, momentum=0.9),
              loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# diff with bottleneck, we have to use data augmentation here
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      vertical_flip = True,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1. / 255) # not in the val data

val_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
import time
# fine-tune the model

ti_ftuning = time.time()


historical_data = model_total.fit_generator(
    train_generator,
    samples_per_epoch=n_train_samples,
    epochs=n_epochs,
    verbose = 1,
    validation_data=val_generator,
    validation_steps=n_validation_samples)

tf_ftuning = time.time()
tt_ftuning = tf_ftuning - ti_ftuning

In [ ]:
print(time.strftime("%H:%M:%S", time.gmtime(tt_ftuning)))

In [ ]:
model_total.save_weights('../data/output_convnet/Inception/inception_ft_model_aux.h5') # testing
# model_total.save_weights('../data/output_convnet/Inception/inception_ft_model.h5')

In [ ]:
steps = int(math.ceil(n_validation_samples / batch_size))
(loss, acc) = model_total.evaluate_generator(val_generator, steps=steps)
print("acc: {0:.2f}% - loss: {1:f}".format(acc * 100, loss))

In [ ]:
train_acc = historical_data.history['acc']
train_loss = historical_data.history['loss']

val_acc = historical_data.history['val_acc'] # validation
val_loss = historical_data.history['val_loss'] # validation

print("train acc mean: {0:.2f} * train loss mean: {1:.2f}".format(np.average(train_acc),np.average(train_loss)))
print("validation acc mean: {0:.2f} * validation loss mean: {1:.2f}".format(np.average(val_acc),np.average(val_loss)))

In [ ]:
# historical_data.history acc and loss data over the epochs (train and validation)

range_epochs = range(n_epochs)

fig = plt.figure(figsize=(20, 6))

fig.add_subplot(1,2,1)
plt.plot(range_epochs, train_acc, 'bo', label='Training acc')
plt.plot(range_epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

# plt.figure()
fig.add_subplot(1,2,2)
plt.plot(range_epochs, train_loss, 'bo', label='Training loss')
plt.plot(range_epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.savefig(os.path.join(PATH_TO_DF, "VGG19_ft_testing_acc_loss.png"), bbox_inches='tight')
# plt.savefig(os.path.join(PATH_TO_DF, "VGG19_ft_acc_loss.png"), bbox_inches='tight')
plt.show()

In [ ]:
# # import h5py

# # hdf5_file = h5py.File('../data/output_convnet/VGG19/VGG19_ft_model.h5', mode='r')

# model_base = InceptionV3(weights='imagenet', include_top=False, input_tensor=input_tensor)  

# # top model
# top_model = Sequential()
# # model.add(Flatten(input_shape=train_data.shape[1:]))
# top_model.add(GlobalAveragePooling2D(input_shape=train_data.shape[1:]))
# top_model.add(Dense(1024, activation='relu'))
# top_model.add(Dense(8, activation='softmax'))

# model_total = Model(input= base_model.input, output= top_model(base_model.output))

# # loading the weights trained before
# model_total.load_weights("../data/output_convnet/Inception/inception_ft_model.h5")

# model_total.compile(optimizer=SGD(lr=1e-4, momentum=0.9),
#               loss='categorical_crossentropy', metrics=['accuracy']) 

In [ ]:
test_convnet = ImageDataGenerator(rescale=1. / 255)

test_generator = test_convnet.flow_from_directory(
    test_data_dir,
    target_size = (img_width, img_height),
    batch_size=batch_size,
    shuffle =False,
    class_mode='categorical')

In [ ]:
steps = int(math.ceil(n_test_samples / batch_size))

(loss, acc) = model_total.evaluate_generator(test_generator, steps=steps)

print("acc: {0:.2f}% - loss: {1:f}".format(acc * 100, loss))

In [ ]:
predictions = model_total.predict_generator(test_generator, steps = steps)

In [ ]:
from sklearn.metrics import confusion_matrix

PATH_TO_IMG = "../data/input_dataset"

prediction_list = []
real_label_list = []

cat_dict = test_generator.class_indices # the key attribute
inverse_coding = {value: key for key, value in cat_dict.items()} # dict of categories

for label in test_generator.classes:
    real_label_list.append(inverse_coding[label])
    
for prediction in predictions:
    prediction_list.append(inverse_coding[np.argmax(prediction)])

# in order to get the confusion matrix
class_names = sorted([folder for folder in os.listdir(PATH_TO_IMG)
                  if os.path.isdir(os.path.join(PATH_TO_IMG, folder))])
prediction_list_arr = np.array(prediction_list) 
real_label_list_arr = np.array(real_label_list)

inception_cm_ft = confusion_matrix(real_label_list_arr,prediction_list_arr)

In [ ]:
# reference http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label',size=20)
    plt.xlabel('Predicted label',size=20)

In [ ]:
fig = plt.figure(figsize=(20, 6))


fig.add_subplot(1,2,1)
plot_confusion_matrix(inception_cm_ft, classes=class_names,
                      title='Confusion matrix, without normalization')

fig.add_subplot(1,2,2)
plot_confusion_matrix(inception_cm_ft, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.savefig(os.path.join(PATH_TO_DF, "inception_ft_test.png"), bbox_inches='tight') # png 70kb vs jpg 135 kb

plt.show()